In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

#Import Libraries
import tensorflow as tf
import numpy as np
import math
import os
import matplotlib
import matplotlib.pyplot as plt
import datetime
import time

import keras
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.utils.vis_utils import plot_model

from sklearn.metrics import recall_score, precision_score
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from models import save_model, generate_model_CNN, plot_performance
from generators import generator, decode

Using TensorFlow backend.


In [2]:
save_dir = 'plots'
if not os.path.exists(save_dir):
        os.makedirs(save_dir)

# X_load = np.load('../data/processed_data/m4019_session1/X.npy')
# Y_load_flat = np.load('../data/processed_data/m4019_session1/y.npy')


# enc = OneHotEncoder(categories='auto')
# Y_load = enc.fit_transform(Y_load_flat.reshape(-1, 1) ,).toarray()

# split_index = int(X_load.shape[0]*0.8)
    
# X_train = X_load[:split_index]
# Y_train = Y_load[:split_index]
# # V_train = V_load[:split_index]

# X_val = X_load[split_index:]
# Y_val = Y_load[split_index:]
# # V_val = V_load[split_index:]


X_train = np.load('../data/processed_data/X_train.npy')
Y_train_flat = np.load('../data/processed_data/y_train.npy')

enc = OneHotEncoder(categories='auto')
Y_train = enc.fit_transform(Y_train_flat.reshape(-1, 1) ,).toarray()


X_val = np.load('../data/processed_data/X_test.npy')
Y_val_flat = np.load('../data/processed_data/y_test.npy')

enc = OneHotEncoder(categories='auto')
Y_val = enc.fit_transform(Y_val_flat.reshape(-1, 1) ,).toarray()

In [3]:
batch_size = 500
time_window = 100
n_dim = 1
class_weight = {0: 1., 1: 50.,}
predict_early = 0

input_shape = (time_window, n_dim)

In [4]:
train_generator = generator(X_train, Y_train, batch_size, time_window, predict_early=predict_early)
validate_generator = generator(X_val, Y_val, batch_size, time_window, predict_early=predict_early)

In [5]:
model = generate_model_CNN(input_shape)

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 1)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 100, 20)           160       
_________________________________________________________________
average_pooling1d_1 (Average (None, 50, 20)            0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 50, 20)            80        
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 50, 20)            2820      
_________________________________________________________________
average_pooling1d_2 (Average (None, 25, 20)            0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 25, 20)            80        
__________

In [6]:
history = model.fit_generator(train_generator,
                              steps_per_epoch=1000,
                              epochs=15, 
                              verbose=1,
                              validation_data=validate_generator,
                              validation_steps=1000,
                              class_weight=class_weight, )

save_model(model, name=predict_early)

Epoch 1/15
1000/1000 [==============================] - 126s 126ms/step - loss: 1.1619 - acc: 0.7262 - val_loss: 0.3373 - val_acc: 0.8721
Epoch 2/15
1000/1000 [==============================] - 100s 100ms/step - loss: 1.0212 - acc: 0.7868 - val_loss: 1.7074 - val_acc: 0.2157
Epoch 3/15
1000/1000 [==============================] - 100s 100ms/step - loss: 0.9265 - acc: 0.7998 - val_loss: 0.1808 - val_acc: 0.9583
Epoch 4/15
1000/1000 [==============================] - 103s 103ms/step - loss: 0.8509 - acc: 0.8074 - val_loss: 0.1285 - val_acc: 0.9589
Epoch 5/15
1000/1000 [==============================] - 104s 104ms/step - loss: 0.8165 - acc: 0.8122 - val_loss: 0.1449 - val_acc: 0.9593
Epoch 6/15
1000/1000 [==============================] - 109s 109ms/step - loss: 0.9008 - acc: 0.8105 - val_loss: 2.3161 - val_acc: 0.1061
Epoch 7/15
1000/1000 [==============================] - 103s 103ms/step - loss: 0.7858 - acc: 0.8174 - val_loss: 0.9377 - val_acc: 0.5017
Epoch 8/15
1000/1000 [============

In [7]:
plot_performance(history)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
validate_generator = generator(X_val, Y_val, batch_size*100, time_window)

In [9]:
X_trial, y_trial, = next(validate_generator)
res = model.predict(X_trial)

In [10]:
threshold=0.9999
y_true = decode(y_trial, threshold=threshold)
y_pred = decode(res, threshold=threshold)

weights = np.zeros_like(y_true)
weights[y_true==0] = 1
weights[y_true==1] = 100


In [11]:
from sklearn.metrics import balanced_accuracy_score, recall_score, classification_report, confusion_matrix

print(balanced_accuracy_score(y_true, y_pred, adjusted=True))
print(recall_score(y_true, y_pred, average='weighted'))

target_names = ['no ripple', 'ripple',]
print(classification_report(y_true, y_pred, target_names=target_names))

0.0014245014245013454
0.95794
              precision    recall  f1-score   support

   no ripple       0.96      1.00      0.98     47894
      ripple       1.00      0.00      0.00      2106

   micro avg       0.96      0.96      0.96     50000
   macro avg       0.98      0.50      0.49     50000
weighted avg       0.96      0.96      0.94     50000



In [12]:
conv_matrix = confusion_matrix(y_true, y_pred)
print(conv_matrix)
conv_matrix = conv_matrix.astype('float') / conv_matrix.sum(axis=1)[:, np.newaxis]
print(conv_matrix)

[[47894     0]
 [ 2103     3]]
[[1.        0.       ]
 [0.9985755 0.0014245]]


In [13]:
plt.figure()
plt.plot(y_true)
plt.plot(y_pred+1)
plt.show()

<IPython.core.display.Javascript object>

In [14]:
plt.figure()
plt.plot(X_load)
plt.plot(Y_load_flat)
plt.show()

<IPython.core.display.Javascript object>

NameError: name 'X_load' is not defined

In [ ]:
Y_load.shape

In [ ]:
# from sklearn.model_selection import TimeSeriesSplit
# tscv = TimeSeriesSplit(n_splits=5)
# for train_index, test_index in tscv.split(X):
#     print("TRAIN:", train_index, "TEST:", test_index)